In [6]:
import pickle
def save_dict(dictionary, path ):
    with open(path, 'wb') as fp:
        pickle.dump(dictionary, fp)

def load_dict(path ):
    with open(path, 'rb') as f:
        return pickle.load(f)

In [1]:
import pandas as pd

df = pd.read_csv('glove.840B.300d.txt', sep=" ", quoting=3, header=None, index_col=0)
glove = {key: val.values for key, val in df.T.items()}

save_dict(glove,'glove.840B.300d.pkl')

In [3]:
glove_dict = load_dict('glove.840B.300d.pkl')

In [5]:
glove_dict[',']

array([-0.082752 ,  0.67204  , -0.14987  , -0.064983 ,  0.056491 ,
        0.40228  ,  0.0027747, -0.3311   , -0.30691  ,  2.0817   ,
        0.031819 ,  0.013643 ,  0.30265  ,  0.0071297, -0.5819   ,
       -0.2774   , -0.062254 ,  1.1451   , -0.24232  ,  0.1235   ,
       -0.12243  ,  0.33152  , -0.006162 , -0.30541  , -0.13057  ,
       -0.054601 ,  0.037083 , -0.070552 ,  0.5893   , -0.30385  ,
        0.2898   , -0.14653  , -0.27052  ,  0.37161  ,  0.32031  ,
       -0.29125  ,  0.0052483, -0.13212  , -0.052736 ,  0.087349 ,
       -0.26668  , -0.16897  ,  0.015162 , -0.0083746, -0.14871  ,
        0.23413  , -0.20719  , -0.091386 ,  0.40075  , -0.17223  ,
        0.18145  ,  0.37586  , -0.28682  ,  0.37289  , -0.16185  ,
        0.18008  ,  0.3032   , -0.13216  ,  0.18352  ,  0.095759 ,
        0.094916 ,  0.008289 ,  0.11761  ,  0.34046  ,  0.03677  ,
       -0.29077  ,  0.058303 , -0.027814 ,  0.082941 ,  0.1862   ,
       -0.031494 ,  0.27985  , -0.074412 , -0.13762  , -0.2186

In [11]:
from main import load_data
train_path = './essay_dataset/training_set_rel3.tsv'
train_df = load_data(train_path)

In [12]:
train_df = train_df[['essay_id','essay_set','essay','rater1_domain1','rater2_domain1','domain1_score']]
set_3 = train_df[train_df.essay_set == 6]

In [14]:
train_df.essay

0        Dear local newspaper, I think effects computer...
1        Dear @CAPS1 @CAPS2, I believe that using compu...
2        Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...
3        Dear Local Newspaper, @CAPS1 I have found that...
4        Dear @LOCATION1, I know having computers has a...
5        Dear @LOCATION1, I think that computers have a...
6        Did you know that more and more people these d...
7        @PERCENT1 of people agree that computers make ...
8        Dear reader, @ORGANIZATION1 has had a dramatic...
9        In the @LOCATION1 we have the technology of a ...
10       Dear @LOCATION1, @CAPS1 people acknowledge the...
11       Dear @CAPS1 @CAPS2 I feel that computers do ta...
12       Dear local newspaper I raed ur argument on the...
13       My three detaileds for this news paper article...
14       Dear, In this world today we should have every...
15       Dear @ORGANIZATION1, The computer blinked to l...
16       Dear Local Newspaper, I belive that computers .

In [ ]:
from string import punctuation

dict_set = set()
for e in train_df.essay:
    r = ''.join([c for c in e.lower() if not c in punctuation])
    dict_set = dict_set.union(r.split())

In [7]:
import numpy as np
scores = np.zeros((5,1))
essays = np.ones((5,4))
b = np.hstack((scores,essays))
b.tolist()

[[0.0, 1.0, 1.0, 1.0, 1.0],
 [0.0, 1.0, 1.0, 1.0, 1.0],
 [0.0, 1.0, 1.0, 1.0, 1.0],
 [0.0, 1.0, 1.0, 1.0, 1.0],
 [0.0, 1.0, 1.0, 1.0, 1.0]]

In [23]:
np.hstack(([[1,1,1],[7,8,9]], [[1,2,3],[4,5,6]]))

array([[1, 1, 1, 1, 2, 3],
       [7, 8, 9, 4, 5, 6]])

In [14]:
a = [[5]]*3
b = [[1,2,3],[1,2,3],[1,2,3]]
a+b

[[5], [5], [5], [1, 2, 3], [1, 2, 3], [1, 2, 3]]

In [12]:
np.repeat(1,4).reshape((4,1))

array([[1],
       [1],
       [1],
       [1]])

In [24]:
[0]+[1]*5

[0, 1, 1, 1, 1, 1]

In [64]:
eos = [0] * 5
eos[0] = -1
a = [[1,2,3,4,5],[7,8,91,10,11]]
a += 2 * [eos]
a

[[1, 2, 3, 4, 5], [7, 8, 91, 10, 11], [-1, 0, 0, 0, 0], [-1, 0, 0, 0, 0]]

In [19]:
import torch
import pandas as pd
from string import punctuation
def load_data(fname):
    # TODO: From the csv file given by filename and return a pandas DataFrame of the read csv.
    df = pd.read_table(fname, encoding = "ISO-8859-1")
    return df
def process_train_data(text, score, word_dict,device):
    # TODO: Input is a pandas DataFrame and return a numpy array (or a torch Tensor/ Variable)
    # that has all features (including characters in one hot encoded form).    
    lowered = ''.join([(' '+c+' ') if c in punctuation else c for c in text.lower() ])
    sequence = lowered.split()
        
    sequence_feats = torch.zeros(len(sequence)+2, 302, dtype=torch.float, device=device)
    sequence_feats[:,0] = float(score)
    
    # SOS is [-1, 300 * 0]
    sequence_feats[0,1] = -1
    
    for ind, word in enumerate(sequence):
        if word not in word_dict:
            word = ''.join([c for c in word if not c.isdigit()])
        if word not in word_dict:
            sequence_feats[ind+1, 1] = 1
            for i,c in enumerate(word):
                sequence_feats[2 + ord(c)] += (i+1) * 0.05 - 0.5
        else:
            sequence_feats[ind+1, 1] = 0
            sequence_feats[ind+1, 2:] = torch.FloatTensor(word_dict[word].tolist(),device=device)
        
    # EOS is [-1, 300 * 1]
    sequence_feats[-1,1:] = -1
    
    #print(np.asarray(essay_feats)[:,0])
    #print(essay_feats[len(essay_feats)-1])
    
    return sequence_feats

In [7]:
train_data_fname = './essay_dataset/training_set_rel3.tsv'

train_data = load_data(train_data_fname) 
train_data.dropna(subset=['domain1_score', 'essay'],how='any',inplace = True)
        
scores = train_data['domain1_score']
essays = train_data['essay']

glove_dict = load_dict('glove.840B.300d.pkl')

#limit = float('inf')
#if len(sys.argv) > 2:
#    limit = int(sys.argv[2])
limit = 10

In [21]:

for index, es in enumerate(essays):
    print(type(scores[index]))
    arr = process_train_data(es, scores[index],glove_dict, 'cuda')
    res= arr
    if (index + 1) % 5000 == 0:
        print(index + 1)
    if index + 1 == limit:
        break
    break
    #print(res)

print(res)

<class 'numpy.int64'>
tensor([[ 8.0000, -1.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 8.0000,  0.0000, -0.2200,  ...,  0.0854, -0.6781, -0.2568],
        [ 8.0000,  0.0000,  0.2165,  ..., -0.9390, -0.1228,  0.6084],
        ...,
        [ 8.0000,  0.0000,  0.3519,  ...,  0.3619, -0.1327, -0.0909],
        [ 8.0000,  0.0000,  0.0120,  ...,  0.1387, -0.3605, -0.0350],
        [ 8.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]], device='cuda:0')


In [1]:
import numpy as np
load = np.load('train.npy')

FileNotFoundError: [Errno 2] No such file or directory: 'train.npy'

In [43]:
np.asarray(load[2]).shape

(319, 302)

In [34]:
print(load[0])

[[ 8.       -1.        0.       ...  0.        0.        0.      ]
 [ 8.        0.       -0.22003  ...  0.085375 -0.67812  -0.25684 ]
 [ 8.        0.        0.21647  ... -0.93896  -0.12281   0.60839 ]
 ...
 [ 8.        0.        0.35185  ...  0.36189  -0.13267  -0.090906]
 [ 8.        0.        0.012001 ...  0.13871  -0.36049  -0.035   ]
 [ 8.       -1.        1.       ...  1.        1.        1.      ]]


In [1]:
import torch 
import numpy as np

train = torch.zeros(3,3,3)

In [39]:
seq1 =[[0,0,1],[2,2,2],[6,6,6]]
seq2 = [[4,5,6],[7,3,9],[6,7,9]]

In [44]:
seqs = [seq1,seq2]
lens = [[3],[2]]
train[[0,1]] = torch.FloatTensor(seqs)
train

tensor([[[ 0.,  0.,  1.],
         [ 2.,  2.,  2.],
         [ 6.,  6.,  6.]],

        [[ 4.,  5.,  6.],
         [ 7.,  3.,  9.],
         [ 6.,  7.,  9.]],

        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]]])

In [5]:
label = torch.zeros(3,1)
label[[0,1,2],0] = torch.FloatTensor([0,1,2])
label.size()

torch.Size([3, 1])

In [33]:
a = [0] * 10
[a]*2

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
import torch
n_indices = torch.zeros((self.batch_size, 1), dtype=torch.long, device=self.device)
batch_arange = torch.arange(self.batch_size, dtype=torch.long, device=self.device)
n_indices[batch_arange,0] = batch_arange

In [6]:
import torch
label = torch.zeros(3, 1, dtype=torch.float)

In [9]:
batch_arange = torch.arange(3,dtype=torch.long)
label[batch_arange, 0] = torch.tensor([0.1,0.5,0.7])

In [11]:
label.size()

torch.Size([3, 1])

In [4]:
import torch
label = torch.zeros(3, 1, dtype=torch.float, device='cuda')

In [5]:
label

tensor([[ 0.],
        [ 0.],
        [ 0.]], device='cuda:0')